In [ ]:
class ParticleSwarmOptimizer:
    def __init__(self, model, X, y, num_particles=10, max_iter=100, c1=2.0, c2=2.0, w=0.8, verbose=True):
        self.model = model
        self.X = X
        self.y = y
        self.num_particles = num_particles
        self.max_iter = max_iter
        self.c1 = c1
        self.c2 = c2
        self.w = w
        self.verbose = verbose

    def optimize_weights(self):
        particles = []
        for i in range(self.num_particles):
            particle_weights = self.model.get_weights()
            particle = {'position': particle_weights, 'velocity': [np.zeros_like(w) for w in particle_weights]}
            particles.append(particle)
        global_best_position = None
        global_best_fitness = float('inf')
        for iteration in range(self.max_iter):
            for particle in particles:
                particle_fitness = self._evaluate_fitness(particle['position'])
                if particle_fitness < global_best_fitness:
                    global_best_position = particle['position']
                    global_best_fitness = particle_fitness
                for i, (w, v, p, g) in enumerate(zip(particle_weights, particle['velocity'], particle['position'], global_best_position)):
                    v = self.w * v + self.c1 * np.random.rand() * (p - w) + self.c2 * np.random.rand() * (g - w)
                    particle['velocity'][i] = v
                    particle['position'][i] = w + v
            if self.verbose:
                print('Iteration {}: global best fitness = {}'.format(iteration, global_best_fitness))
        self.model.set_weights(global_best_position)

    def _evaluate_fitness(self, weights):
        self.model.set_weights(weights)
        loss = log_loss(self.y, self.model.predict(self.X))
        return loss